
# Reading and Writing Aim Files Tutorial

In this tutorial, we will go through the process of reading, processing, and saving an AIM file (Scanco) using `vtkbone` and `vtk`. We will apply a Gaussian filter to smooth the image and then save the processed image along with its processing log.


In [4]:

import vtk
import vtkbone
import numpy as np
import sys
import os

# Find the path to the 'python_helpers' directory
notebook_directory = os.getcwd()  # Gets the current working directory where the notebook is
helpers_path = os.path.join(notebook_directory, 'python_helpers')

# Add this directory to sys.path
if helpers_path not in sys.path:
    sys.path.append(helpers_path)

# Now you can import your module or functions
from format_aim_header import logtodict, dicttolog

## Setup and Read the Image

First, we'll read an image from a given file. We'll use `vtkbone.vtkboneAIMReader` for this purpose.

In [5]:

input_aim = '/Users/matthiaswalle/Downloads/IMG1237.AIM'

print(f'Reading file: {input_aim}')
reader = vtkbone.vtkboneAIMReader()
reader.DataOnCellsOff()
reader.SetFileName(input_aim)
reader.Update()
image = reader.GetOutput()

Reading file: /Users/matthiaswalle/Downloads/IMG1237.AIM



## Process and Print the Log

We will extract the processing log from the image and then make some modifications to it.


In [7]:

processing_log = reader.GetProcessingLog()
processing_log_dict = logtodict(processing_log)
print(processing_log)


# Add to the processing log
processing_log_dict['Created by'] = 'N88_VTKBONE'
processing_log_dict['Original Density: intercept'] = processing_log_dict['Density: intercept']
processing_log_dict['Density: intercept'] = -400



!
! Processing Log
!
!-------------------------------------------------------------------------------
Created by                    ISQ_TO_AIM (IPL)                                  
Time                          16-APR-2017 20:20:59.67                           
Original file                 dk0:[xtremect2.data.00001237.00005697]d0005679.isq;
Original Creation-Date        14-SEP-2016 15:27:12.18                           
Orig-ISQ-Dim-p                                   2304       2304        168
Orig-ISQ-Dim-um                                139852     139852      10197
!-------------------------------------------------------------------------------
Patient Name                  DBQ_152                                           
Index Patient                                    1237
Index Measurement                                5697
!-------------------------------------------------------------------------------
Site                                               20
Scanner ID      


## Apply Gaussian Smoothing

Next, we'll apply a Gaussian smoothing filter to the image. This is a common preprocessing step in image analysis.


In [8]:

processing_log_dict['Gauss Filter (sigma)'] = 1.2

gaussian_smooth = vtk.vtkImageGaussianSmooth()
gaussian_smooth.SetInputData(image)
gaussian_smooth.SetStandardDeviation(1.2)  
gaussian_smooth.Update()
output_image = gaussian_smooth.GetOutput()
        


## Save the Processed Image

Finally, we will save the processed image and its updated processing log.


In [13]:

output_aim = '/Users/matthiaswalle/Downloads/IMG1237_gauss_logged.AIM'
writer = vtkbone.vtkboneAIMWriter()
writer.SetInputData(output_image)
writer.SetFileName(output_aim)
writer.NewProcessingLogOff()
writer.SetProcessingLog(dicttolog(processing_log_dict))
writer.Update()



## Test the Result

As a final step, let's read back the saved image and print its processing log to ensure everything went as expected.


In [14]:

reader = vtkbone.vtkboneAIMReader()
reader.DataOnCellsOff()
reader.SetFileName(output_aim)
reader.Update()
image = reader.GetOutput()

processing_log = reader.GetProcessingLog()
print(processing_log)
        

! Processing Log
!
!-------------------------------------------------------------------------------
Created by                    N88_VTKBONE                                       
Time                          16-APR-2017 20:20:59.67                           
Original file                 dk0:[xtremect2.data.00001237.00005697]d0005679.isq;
Original Creation-Date        14-SEP-2016 15:27:12.18                           
Orig-ISQ-Dim-p                      2304       2304        168
Orig-ISQ-Dim-um                   139852     139852      10197
!-------------------------------------------------------------------------------
Patient Name                  DBQ_152                                           
Index Patient                                    1237
Index Measurement                                5697
!-------------------------------------------------------------------------------
Site                                               20
Scanner ID                                  